In [1]:
%load_ext autoreload
%autoreload 2

import ablang

In [2]:
model = ablang.pretrained("/vols/bitbucket/olsen/models/ABLANG-709/")

In [25]:
seq1 = {
    'H':'EVQLLESGGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS',
    'L':'DIQLTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK'
}
seq2 = {
    'H':'EVQLLGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS',
    'L':'DIQLTQSPLSLPVTLGQPASISCRSSQSLEASDTNIYLSWFQQRPGQSPRRLIYKISNRDSGVPDRFSGSGSGTHFTLRISRVEADDVAVYYCMQGTHWPPAFGQGTKVDIK'
}
seq3 = {
    'H':'EVQLLGEVKKPGASVKVSCRASGYTFRNYGLTWVRQAPGQGLEWMGWISAYNGNTNYAQKFQGRVTLTTDTSTSTAYMELRSLRSDDTAVYFCARDVPGHGAAFMDVWGTGTTVTVSS',
    }

seqs = [seq1, seq2]

In [140]:
results = model(seqs, mode='rescoding', align=True)
results

[aligned_results(aligned_embeds=array([[[-0.40741199254989624, -0.5118988752365112, 0.06096703186631203,
          ..., 0.03920229896903038, -0.36715835332870483, '<'],
         [-0.5768881440162659, 0.3824540078639984, -0.21792010962963104,
          ..., -0.08844482898712158, -0.3236753046512604, 'E'],
         [-0.14759314060211182, 0.3963906466960907, -0.38226985931396484,
          ..., -0.41469550132751465, -0.0031935935840010643, 'V'],
         ...,
         [-0.1435835361480713, 0.3124387264251709, -0.3015800714492798,
          ..., -0.4535340368747711, -0.07878872752189636, 'I'],
         [0.17538957297801971, 0.24394312500953674, 0.2014114260673523,
          ..., -0.3847905099391937, 0.07409171760082245, 'K'],
         [-0.23031717538833618, -0.35487303137779236, 0.1960681825876236,
          ..., 0.3110731840133667, -0.3265109062194824, '>']],
 
        [[-0.28587281703948975, 0.10260795056819916, 0.48066675662994385,
          ..., -0.25312963128089905, 0.0360877811908721

In [100]:
len(seq1['H']) + len(seq1['L'])

233

In [101]:
results[0].shape

AttributeError: 'aligned_results' object has no attribute 'shape'

In [141]:
results[0].aligned_embeds.shape

(2, 238, 481)

In [126]:
results[0].number_alignment.shape

(238,)

In [139]:
import numpy as np

np.concatenate([results[0].aligned_embeds], axis=0).shape

(2, 238, 481)